In [120]:
import numpy as np
import pandas as pd
from tslearn.metrics import dtw, lb_keogh
from sklearn.metrics import accuracy_score

# from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.metrics import dtw as tsln_dtw

# from tqdm import tqdm
from aeon.utils.numba.general import z_normalise_series
from aeon.distances import get_distance_function

In [121]:
%load_ext autoreload
%autoreload 2

from ksfdtw.utils import precision_at_k

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
hello_world()

Hello, world! 2


In [123]:
# # A neat way to load the dataset
# data = np.load("../data/gunpoint_preprocessed.npz")
# data_dict = {key: data[key] for key in data.files}

In [130]:
# A old way to load the dataset
data = np.load(
    "../data_intermediate/GunPoint_preprocessed_P_3_l_2.0_len_150.npz",
    allow_pickle=True,
)
X_train_scaled = data["X_train_scaled"]
X_train_proc = data["X_train_proc"]
X_train_proc_noise = data["X_train_proc_noise"]
y_train = data["y_train"]
X_test_scaled = data["X_test_scaled"]
X_test_proc = data["X_test_proc"]
X_test_proc_noisy = data["X_test_proc_noise"]
y_test = data["y_test"]
train_cutting_orig = data["train_cutting_orig"].tolist()
train_cutting_final = data["train_cutting_final"].tolist()
test_cutting_orig = data["test_cutting_orig"].tolist()
test_cutting_final = data["test_cutting_final"].tolist()

# Precision@k

Compute $P@k$ for querying $Q \in \mathcal{D}_{\text{train, ps}}$ using DTW

In [134]:
# DTW
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0, 50):
    distances = np.array(
        [
            tsln_dtw(
                z_normalise_series(X_train_proc[i]), z_normalise_series(x.squeeze())
            )
            for x in X_train_scaled
        ]
    )
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(
    precision_at_1 / 50, precision_at_3 / 50, precision_at_5 / 50, precision_at_7 / 50
)
# 0.48 0.74 0.86 0.9

18 22 31 35
0.36 0.44 0.62 0.7


Compute $P@k$ for querying $Q \in \mathcal{D}_{\text{train, ps, noise}}$ using DTW

In [20]:
# DTW Noise
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0, 50):
    distances = np.array([tsln_dtw(X_train_proc_noise[i], x) for x in X_train_scaled])
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(
    precision_at_1 / 50, precision_at_3 / 50, precision_at_5 / 50, precision_at_7 / 50
)
# 0.44 0.74 0.8 0.88

30 38 46 49
0.6 0.76 0.92 0.98


Compute $P@k$ for querying $Q \in \mathcal{D}_{\text{train, ps}}$ using ED

In [21]:
# ED
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0, 50):
    distances = np.array(
        [euclidean_distance(X_train_proc[i], x) for x in X_train_scaled]
    )
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(
    precision_at_1 / 50, precision_at_3 / 50, precision_at_5 / 50, precision_at_7 / 50
)
# 0.02 0.06 0.1 0.18

NameError: name 'euclidean_distance' is not defined

Compute $P@k$ for querying $Q \in \mathcal{D}_{\text{train, ps, noise}}$ using ED

In [188]:
# ED
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0, 50):
    distances = np.array(
        [euclidean_distance(X_train_proc_noise[i], x) for x in X_train_scaled]
    )
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(
    precision_at_1 / 50, precision_at_3 / 50, precision_at_5 / 50, precision_at_7 / 50
)
# 0.02 0.06 0.08 0.18

2 4 6 7
0.04 0.08 0.12 0.14


In [101]:
from aeon.distances import euclidean_distance
from aeon.distances import dtw_distance

In [111]:
target_set = X_train_proc_noise

In [129]:
dist_func = get_distance_function("euclidean")
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0, 50):
    distances = np.array(
        [
            dist_func(
                z_normalise_series(target_set[i]),
                z_normalise_series(np.squeeze(x)),
                # z_normalise_series(target_set[i]),
                # z_normalise_series(np.squeeze(x)),
            )
            for x in X_train_scaled
        ]
    )
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(
    precision_at_1 / 50, precision_at_3 / 50, precision_at_5 / 50, precision_at_7 / 50
)
# 0.02 0.06 0.08 0.18

1 3 5 7
0.02 0.06 0.1 0.14


# Results
target_set = X_train_proc
```
euclidean 0.18 0.38 0.52 0.6
dtw 0.24 0.4 0.56 0.64
adtw 0.2 0.34 0.48 0.64
ddtw 0.2 0.42 0.52 0.58
shape_dtw 0.22 0.26 0.38 0.56
wddtw 0.2 0.42 0.54 0.62
wdtw 0.24 0.4 0.58 0.64
```
target_set = X_train_proc_noise
```
euclidean 0.16 0.34 0.54 0.56
dtw 0.24 0.44 0.5 0.6
adtw 0.18 0.34 0.52 0.66
ddtw 0.24 0.38 0.46 0.5
shape_dtw 0.22 0.32 0.44 0.58
wddtw 0.26 0.42 0.48 0.56
wdtw 0.22 0.44 0.5 0.64
```

In [88]:
from aeon.distances import get_distance_function
import numpy as np

x = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
y = np.array([[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
dtw_dist_func = get_distance_function("dtw")
dtw_dist_func(x, y)

768.0

In [85]:
import numpy as np
from aeon.distances import dtw_distance

x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = np.array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
dtw_distance(x, y)  # 1D series

768.0